In [4]:
###### import packages, sentence model, and parse html from site url
import requests
import re
from bs4 import BeautifulSoup
import jieba
import torch
import string
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
sentence_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
language_model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')

url = 'https://www.bbc.com/zhongwen/articles/c4gl97d2rzjo/simp'
site = requests.get(url)
site_soup = BeautifulSoup(site.text, 'html.parser')
# print(site_soup.prettify())

In [87]:
HSK = pd.read_pickle("../data/Chinese/HSK_full")
HSK.head()

,character,POS,pinyin,HSK,definition,embedding
0,爱,V,ài,1,"love, like, be fond of, be keen on, cherish, b...","[0.37996447, 0.29299688, 0.55704415, 0.0736245..."
1,爱好,V/N,àihào,1,"love, like, be fond of, be keen on","[0.4404698, 0.29555953, 0.5669606, -0.0190896,..."
2,爸爸,NaN,bàba,1,"old man, father, papa, pappa, daddy, pa, beget...","[0.05222196, 0.18916288, 0.2597713, -0.585589,..."
3,白,Adj,bái,1,"white, clear, pure, plain, wrongly written/mis...","[-0.17480375, 0.23118941, -0.27152503, 0.10098..."
4,白,Adv,bái,1,"white, clear, pure, plain, wrongly written/mis...","[-0.17480375, 0.23118941, -0.27152503, 0.10098..."


In [14]:
site_p[20].string

'演讲的视频和文字一度在社交媒体上疯传，但后来却从中国互联网上消失。根据路透社和中国媒体的报道，上周高善文的微信帐号因“违反使用规范”无法关注，外界无法透过微信与他取得联络。东北证券首席经济学家付鹏的微信视频号也被禁止关注。'

In [21]:
###### get text from the html and tokenize
site_p = site_soup.find_all('p')
text = ""
punc = "！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏." # possible punctuation

for p in site_p:
    text += str(p.get_text()) # append new text to the text string

text.translate(str.maketrans('', '', string.punctuation)) # convert str format
text_re = re.sub(r"[%s]+" %punc, "", text) # remove punctuation marks

tokens = jieba.lcut(text_re) # tokenize
tokens_l = a = list(dict.fromkeys(tokens))

In [23]:
###### get text from the html and tokenize
site_s = site_soup.find_all(string=True)
text = ""
punc = "！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏." # possible punctuation

for s in site_s:
    text += str(s.get_text()) # append new text to the text string

text.translate(str.maketrans('', '', string.punctuation)) # convert str format
text_re = re.sub(r"[%s]+" %punc, "", text) # remove punctuation marks

tokens = jieba.lcut(text_re) # tokenize
tokens_l = a = list(dict.fromkeys(tokens))

In [88]:
###### Cross compute the top embeddings for each vocab word and add to HSK dataframe
similarity_t = sentence_model.similarity(HSK["embedding"], HSK["embedding"])
top_choice = torch.flip(np.argsort(similarity_t, axis=1)[:,-21:-1], dims=(1,)).numpy().tolist()
top_choice_HSK = [[HSK["HSK"][i] for i in row] for row in top_choice] # convert from index values to HSK values

c:\Users\tempu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\util.py:44: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  a = torch.tensor(a)


In [151]:
def simplify(tokens, max_HSK):
    simplified_tokens = dict()
    for token in tokens:
        try:
            char_idx = np.where(HSK["character"]==token)[0][0]
            if HSK["HSK"][char_idx]>max_HSK:
                top_idx = top_choice[char_idx][next(x[0] for x in enumerate(top_choice_HSK[char_idx]) if x[1] <= max_HSK)] # iterate through and find index of first element below max_HSK
                simplified_tokens[token] = HSK["character"].loc[top_idx]
            else:
                pass# simplified_tokens[idx] = 0
        except:
            pass# simplified_tokens[idx] = 1
    return simplified_tokens

In [164]:
replacement_dict = simplify(tokens_l, 4)

In [179]:
for element in site_soup.find_all(string=True):  # Get all text nodes
    text = element
    for old_word, new_word in replacement_dict.items():
        if old_word in text:
            text = text.replace(old_word, f'<span style="color: red;">{new_word}</span>')
    element.replace_with(BeautifulSoup(text, 'html.parser'))

In [180]:
with open("output.html", "w", encoding = 'utf-8') as file: 
    # prettify the soup object and convert it into a string 
    file.write(str(site_soup.prettify())) 